# Perceptron

This notebook shows the implementation of a perceptron in Python, which structure illustrated bellow.

<center><img src="resources/img/perceptron.png" alt="Perceptron" width="500px"/></center>

Some datasets are evaluated and the results are discussed in the following cells. The main idea is to show in a 2D figure the distringuishing approch of a perception, illustration the lines that separates two classes of a dataset, has illustrated bellow in a 3D surface.

<center><img src="resources/img/linear_classifier.png" alt="Separation line of a linear classifier" width="500px"/></center>

Before we start, it is useful to define some functions first.

### Useful functions

In [ ]:
from numpy        import array, hstack, inner, isscalar, mean, ndim, repeat, shape, squeeze, transpose, vstack, zeros
from numpy.random import MT19937, RandomState, SeedSequence

msqr             = lambda a: mean(inner(a,a))  # mean square
ndims            = lambda a: shape(a)[1];      # number of dimensions
nelems           = lambda a: shape(a)[0];      # number of elements
new_random_state = lambda seed=123: RandomState(MT19937(SeedSequence(seed)))

random_state = new_random_state()

# # Inser a column to a given 2D matrix at specified index
# def insert_column(X, column=[], index=-1):
#     nd = ndim(X)
#     if nd == 0:
#         n = 1
#         m = 1
#     elif nd == 1:
#         n = 1
#         m = len(X)
#     else:
#         n, m = X.shape
#     Y   = zeros((n, m + 1))
#     rng = list(range(m + 1))
#     if index == -1:
#         I = array(rng[0:index])
#     else:
#         I = array(rng[0:index] + rng[index+1:m+1])
#     Y[:,I] = X
#     if isscalar(column):
#         Y[:,index] = column
#     elif len(column) != 0:
#         Y[:,index] = squeeze(column)
#     if n == 1:
#         Y = Y[0]
#     return Y

def display(matrix, nrows=-1):
    if ndim(matrix) <= 1:
        matrix = transpose([matrix])
    if nrows == 0:
        submatrix = []
    elif nrows == -1:
        submatrix = matrix
    elif nrows < -1:
        submatrix = matrix[:nrows+1,:]
    else:
        submatrix = matrix[:nrows,:]
    if shape(submatrix) == shape(matrix):
        out = matrix
    else:
        n    = ndims(matrix)
        dots = repeat(['...'], n)
        end  = matrix[-1,:]
        out  = vstack((submatrix, dots, end))
    print(out)

def display_data(inputs, labels, nrows=-1):
    matrix = hstack([inputs, transpose([labels])])
    display(matrix, nrows)

# Uniform random between a and b
def urand(a, b, *args, **kwargs):
    r   = random_state.rand(*args, **kwargs)
    rab = (b - a) * r + a
    return rab

### Load data function

In [ ]:
# from numpy import array, loadtxt, transpose
from numpy import loadtxt

def load_data1(filename):
    data   = loadtxt(filename)
    inputs = data[:,0:2]
    labels = data[:,2] - 1
    return inputs, labels

def load_data2(filename):
    pass

def load_data3(filename):
    data   = loadtxt(filename, delimiter=';')
    inputs = data[:,1]
    labels = data[:,0]
    return transpose([inputs]), labels
#     return [inputs], labels

def keep_only_labels(inputs, labels, labels_to_keep):
    selected_inputs = array([x.tolist() for x, y in zip(inputs, labels) if y in labels_to_keep])
    selected_labels = array([y for y in labels if y in labels_to_keep])
    return selected_inputs, selected_labels

def keep_only_labels2(inputs, labels):
    labels_to_keep                   = unique(labels)[0:2] # only first two labels
    selected_inputs, selected_labels = keep_only_labels(inputs, labels, labels_to_keep)
    return selected_inputs, selected_labels

### Neuron base class
This cells implements a base neuron class that can be used with eighter a perceptron, an ADALINE or even any other similar structure with a single output.

In [ ]:
from numpy        import append, full, insert, ones, ones_like, sign, unique, where
from numpy.random import permutation

class Neuron():
    def __init__(self, input_length, learning_rate=0.01, max_epochs=1e2, min_loss=1e-2):
        self.input_length  = int(input_length)
        self.learning_rate = learning_rate
        self.max_epochs    = int(max_epochs)
        self.min_loss      = min_loss
        self.weights       = self.__initialize_weight__(length=input_length+1)

        # Propertiers defined elsewhere
        self.iterations    = None
        self.epochs        = None
        self.inputs        = None
        self.labels        = None
        self.losses        = None
        self.unique_labels = None

    def __adapt__(self, input, error, iteration=-1):
        w = self.weights[iteration]
        n = self.learning_rate
        new_weight   = w + n*input*error # adaptation rule
        self.weights = append(self.weights, [new_weight], axis=0)

    def __calc_cost__(self, errors):
        cost = msqr(errors) # mean square error
        return cost

    def __calc_error__(self, labels, outputs):
        diff   = labels - outputs
        errors = sign(diff)
        return errors

    def __converged__(self, iteration=-1):
        if len(self.losses) > 0:
            converged = self.losses[iteration] <= self.min_loss # convergence rule
        else:
            converged = False
        return converged

    def __expand_inputs__(self, inputs):
        num_inputs      = nelems(inputs)
        biases          = -ones((1, num_inputs))
        expanded_inputs = insert(inputs, 0, biases, axis=1)
        return expanded_inputs

    def __initialize_weight__(self, length):
        w0 = urand(-1, 1, length)
#         w0 = [-0.6270591 ,  1.96598375, -1.89546286]
        return [w0]

    def __predict__(self, input, iteration=-1):
        weights       = self.weights[iteration]
        inner_state   = inner(weights, input) # induced local field
        neuron_output = self.__activation_function__(inner_state)
        output        = self.__translate_output__(neuron_output)
        return output.tolist()

    def predict(self, input, iteration=-1):
        expanded_input = self.__expand_inputs__(input)
        output         = self.__predict__(expanded_input, iteration)
        return output

    def train(self, inputs, labels, shuffle_data=False):
        def prepare_training(inputs, labels, shuffle_data):
            self.inputs         = inputs
            self.labels         = labels
            self.unique_labels  = unique(labels)
            self.num_inputs     = nelems(inputs)
            self.max_iterations = self.num_inputs*(self.max_epochs + 1) + 1
            self.losses         = []
            expanded_inputs     = self.__expand_inputs__(inputs)
            if shuffle_data:
                permutations    = permutation(self.num_inputs) # random shuffle
                expanded_inputs = expanded_inputs[permutations]
                self.inputs     = self.inputs[permutations]
                self.labels     = self.labels[permutations]
            return expanded_inputs, self.labels

        def finish_training(iteration):
            self.iterations = iteration
            self.epochs     = int(iteration/self.num_inputs) + 1

        expanded_inputs, labels = prepare_training(inputs, labels, shuffle_data)
        # Calculate loss with all inputs (for initial weights)
        outputs = self.__predict__(expanded_inputs)
        errors  = self.__calc_error__(labels, outputs)
        self.losses.append(self.__calc_cost__(errors))
        # Train loop
        iteration = 0
        while not self.__converged__() and iteration < self.max_iterations:
            for input, label in zip(expanded_inputs, labels):
                # Adapt with given input
                output = self.__predict__(input)
                error  = self.__calc_error__(label, output)
                self.__adapt__(input, error)
                iteration += 1
                # Calculate loss with all inputs (this can be moved to outside of the loop for efficiency)
                outputs = self.__predict__(expanded_inputs)
                errors  = self.__calc_error__(labels, outputs)
                self.losses.append(self.__calc_cost__(errors))
                if self.__converged__() or iteration == self.max_iterations: break # taking advantage of iteration-based neuron
        finish_training(iteration)

### Perceptron class
The Percetron class then inherits the base Neuron class and implements its out activation and output translation functions.

The function `kernel` is useful to determine the separation line between two classes.

The function `predict` is overloaded to process the input if any adaptation is required beforehand.

In [ ]:
class Perceptron(Neuron):
    def __activation_function__(self, inner_state):
        act_fcn = where(inner_state < 0, 0, 1) # all-or-none
        return act_fcn

    def __translate_output__(self, neuron_output):
        classes = self.unique_labels
        output  = where(neuron_output <= 0, classes[0], classes[1])
        return output

    def kernel(self, input_class1=None, input_class2=None, iteration=-1):
        weight = self.weights[iteration]
        if input_class1 is None:
            input_class1 = []
            for x in input_class2:
                input  = self.__expand_inputs__([[0, x]])
                weight = -weight/weight[1]
                input_class1.append(inner(weight, input)[0])
            return input_class1
        elif input_class2 is None:
            input_class2 = []
            for x in input_class1:
                input  = self.__expand_inputs__([[x, 0]])
                weight = -weight/weight[2]
                input_class2.append(inner(weight, input)[0])
            return input_class2
        else:
            return None

    def predict(self, input, iteration=-1):
        if ndim(input) == 1:
            input = [input]
        return super().predict(input, iteration)

### Adaline classes
The Adaline classes also inherits the base Neuron class and implements its out activation, output translation, kernel and predict functions.

#### Linear ADALINE
This Adaline class implements a first order ADALINE regressor.

In [ ]:
class Adaline1(Neuron):
    def __activation_function__(self, inner_state):
        act_fcn = inner_state
        return act_fcn

    def __translate_output__(self, neuron_output):
        output = neuron_output
        return output

#     def kernel(self, partial_input, missing_dimension, iteration=-1):
#         weight       = self.weights[iteration]
#         input_kernel = []
#         for row in partial_input:
#             input           = insert_column(row, index=missing_dimension)
#             expanded_input  = self.__expand_inputs__([input])
#             relative_weight = -weight/weight[missing_dimension]
#             inner_state     = inner(relative_weight, input)
#             if not isscalar(inner_state):
#                 inner_state = inner_state[0]
#             input_kernel.append(inner_state)
#         return input_kernel

    def predict(self, input, iteration=-1):
        if isscalar(input):
            input = transpose([[input]])
            output = super().predict(input, iteration)[0]
            return output
        if ndim(input) == 1:
            input = transpose([input])
        output = super().predict(input, iteration)
        return output

#### 2nd-order ADALINE
This Adaline class implements a second order ADALINE regressor.

In [ ]:
class Adaline2(Neuron):
    def __activation_function__(self, inner_state):
        act_fcn = inner_state
        return act_fcn

    def __translate_output__(self, neuron_output):
        output = neuron_output
        return output

#     def kernel(self, partial_input, missing_dimension, iteration=-1):
#         weight       = self.weights[iteration]
#         input_kernel = []
#         for row in partial_input:
#             input           = insert_column(row, index=missing_dimension)
#             expanded_input  = self.__expand_inputs__([input])
#             relative_weight = -weight/weight[missing_dimension]
#             inner_state     = inner(relative_weight, input)
#             if not isscalar(inner_state):
#                 inner_state = inner_state[0]
#             input_kernel.append(inner_state)
#         return input_kernel

    def predict(self, input, iteration=-1):
        def raise_order(input):
            input = insert(input, 1, values=transpose(input**2), axis=1)
            return input

        if isscalar(input):
            input = transpose([[input]])
            input = raise_order(input)
            output = super().predict(input, iteration)[0]
            return output
        if ndim(input) == 1:
            input = transpose([input])
        if ndims(input) == 1:
            input = raise_order(input)
        output = super().predict(input, iteration)
        return output

### Figure class
The following class can look complex, but it is composed by a set of functions that process the neuron output and convergence, illustrating the results in epochs.

For the reader interested in the perceptron itself, this cell can be skipped.

In [ ]:
from bokeh.io       import output_notebook
from bokeh.layouts  import column, row
from bokeh.models   import ColumnDataSource, CustomJS, LinearAxis, Range1d, Slider, tickers
from bokeh.plotting import Figure, output_file, show
from numpy          import argsort, array, concatenate, sort

output_notebook()

class Figure_classification():
    def __init__(self, neuron, width=400, height=400):
        self.neuron = neuron
        self.__init_plot_params__()

        [fig1, area_left, area_right] = self.__create_figure1__()
        fig2   = self.__create_figure2__()
        slider = self.__create_slider__(area_left, area_right)
        if slider is None:
            self.layout = row(fig1, fig2)
        else:
            self.layout = column(slider, row(fig1, fig2))

    def __create_figure1__(self, width=400, height=400):
        figure     = Figure(plot_width=width, plot_height=height)
        colors_lr  = self.source1_weights.tags[2][-1]
        area_left  = figure.varea(x='x', y1='y1_left',  y2='y2_left',  source=self.source1_weights, fill_color=colors_lr[0], fill_alpha=0.2)
        area_right = figure.varea(x='x', y1='y1_right', y2='y2_right', source=self.source1_weights, fill_color=colors_lr[1], fill_alpha=0.2)
        for class_input, legend_label, color in zip(self.class_input, self.legend_labels, self.colors):
            x = class_input[:,0]
            y = class_input[:,1]
            c = figure.circle(x, y, size=10, fill_color=color, fill_alpha=0.6, line_color=None, legend_label=legend_label)
        figure.line('x', 'y', source=self.source1_weights, line_width=3, line_alpha=0.6, line_color='black')
        figure.circle('x', 'y', source=self.source1_selection, size=10, fill_color=None, line_color='black', line_alpha=0.6, line_width=3)
        # Properties
        figure.x_range = self.x_range
        figure.y_range = self.y_range
        figure.legend.click_policy = 'hide'
        return figure, area_left, area_right

    def __create_figure2__(self, width=400, height=400):
        figure = Figure(plot_width=width, plot_height=height, x_axis_label='Iteration', y_axis_label='Loss')
        # Iteration axis
        x_iteration = self.source2_iteration.data['x']
        y_iteration = self.source2_iteration.data['y']
        figure.line(x_iteration, y_iteration, line_width=3, line_color='black', line_alpha=0.1)
        figure.line('x', 'y', source=self.source2_iteration, line_width=3, line_color='black', legend_label='Iteration loss')
        # Epoch axis
        x_epoch = self.source2_epoch.data['x']
        y_epoch = self.source2_epoch.data['y']
        color_epoch = 'red'
        figure.extra_x_ranges = {'epoch': Range1d(start=0, end=self.neuron.epochs)}
        figure.line(x_epoch, y_epoch, x_range_name='epoch', line_width=3, line_color=color_epoch, line_alpha=0.1)
        figure.line('x', 'y', source=self.source2_epoch, x_range_name='epoch', line_width=3, line_color=color_epoch, legend_label='Epoch loss')
        figure.add_layout(LinearAxis(x_range_name           = 'epoch',
                                     axis_label             = 'Epoch',
                                     axis_label_text_color  = color_epoch,
                                     axis_line_color        = color_epoch,
                                     major_label_text_color = color_epoch,
                                     major_tick_line_color  = color_epoch,
                                     minor_tick_line_color  = color_epoch),
                          'above')
        # Properties
        if min(x_iteration) != max(x_iteration):
            figure.x_range = Range1d(min(x_iteration), max(x_iteration), bounds="auto")
        else:
            figure.x_range = Range1d(min(x_iteration), max(x_iteration) + 1, bounds="auto")
        figure.y_range = Range1d(0, max(y_iteration) + 1, bounds="auto")
        figure.xaxis.ticker.min_interval = 1
        figure.yaxis.ticker.min_interval = 1
        figure.xaxis.ticker.num_minor_ticks = 0
        figure.yaxis.ticker.num_minor_ticks = 0
        figure.legend.click_policy = 'hide'
        return figure

    def __create_slider__(self, area_left, area_right):
        if self.neuron.iterations == 0:
            return None
        else:
            callback1_weights = CustomJS(args=dict(area_left=area_left, area_right=area_right, source=self.source1_weights), code="""
                var i                       = cb_obj.value
                var x1_span                 = source.tags[0][i]
                var x2_span                 = source.tags[1][i]
                var colors_lr               = source.tags[2][i]
                var x2_span_s               = x2_span.slice().sort()
                source.data['x']            = x1_span
                source.data['y']            = x2_span
                source.data['y1_left']      = [x2_span_s[0], x2_span[1]]
                source.data['y2_left']      = [x2_span_s[1], x2_span[1]]
                source.data['y1_right']     = [x2_span[0],   x2_span_s[0]]
                source.data['y2_right']     = [x2_span[0],   x2_span_s[1]]
                area_left.glyph.fill_color  = colors_lr[0]
                area_right.glyph.fill_color = colors_lr[1]
                source.change.emit()
            """)
            callback1_selection = CustomJS(args=dict(source=self.source1_selection), code="""
                var i            = cb_obj.value
                var X            = source.tags[0]
                var Y            = source.tags[1]
                source.data['x'] = [X[i % X.length]]
                source.data['y'] = [Y[i % Y.length]]
                source.change.emit()
            """)
            callback2_iteration = CustomJS(args=dict(source=self.source2_iteration), code="""
                var i            = cb_obj.value
                var x            = source.tags[0].slice(0, i + 1)
                var y            = source.tags[1].slice(0, i + 1)
                source.data['x'] = x
                source.data['y'] = y
                source.change.emit()
            """)
            callback2_epoch = CustomJS(args=dict(source=self.source2_epoch), code="""
                var i    = cb_obj.value
                var end  = cb_obj.end
                var base = source.tags[2]
                if (i !== end)
                {
                    var e = Math.floor(i / base)
                }
                else
                {
                    var e = Math.ceil(i / base)
                }
                var x            = source.tags[0].slice(0, e + 1)
                var y            = source.tags[1].slice(0, e + 1)
                source.data['x'] = x
                source.data['y'] = y
                source.change.emit()
            """)
            slider = Slider(start=0, end=self.neuron.iterations, value=self.neuron.iterations, step=1, title='Iteration')
            slider.js_on_change('value', callback1_weights)
            slider.js_on_change('value', callback1_selection)
            slider.js_on_change('value', callback2_iteration)
            slider.js_on_change('value', callback2_epoch)
            return slider

    def __init_plot_params__(self):
        n  = self.neuron.input_length
        k  = self.neuron.num_inputs
        X  = self.neuron.inputs
        Y  = self.neuron.labels
        C  = self.neuron.unique_labels
        W  = self.neuron.weights
        I  = range(0, self.neuron.iterations+1)
        E  = range(0, self.neuron.epochs+1)
        L  = self.neuron.losses
        LE = L[::k]
        LE.append(L[-1])
        self.colors        = ['blue', 'red']
        self.legend_labels = ['Class ' + str(int(label)) for label in self.neuron.unique_labels]
        x1min, x1max = min(X[:,0]), max(X[:,0])
        x2min, x2max = min(X[:,1]), max(X[:,1])
        dx1,   dx2   = x1max - x1min, x2max - x2min
        x1_range     = [x1min - 0.1*dx1, x1max + 0.1*dx1]
        x2_range     = [x2min - 0.1*dx2, x2max + 0.1*dx2]
        X1_span      = []
        X2_span      = []
        colors_lr    = []
        for iteration in I:
            x1_span = array(x1_range)
            x2_span = array(self.neuron.kernel(input_class1=x1_span, iteration=iteration))
            i       = argsort(x2_span)
            if x2_span[i[0]] > x2_range[0] or x2_span[i[1]] < x2_range[1]:
                if x2_span[i[0]] > x2_range[0]:
                    x2_span[i[0]] = x2_range[0]
                if x2_span[i[1]] < x2_range[1]:
                    x2_span[i[1]] = x2_range[1]
                x1_span = array(self.neuron.kernel(input_class2=x2_span, iteration=iteration))
            X1_span.append(x1_span)
            X2_span.append(x2_span)
            input_left   = [x1_span[0] - 1, x2_span[0]]
            input_right  = [x1_span[0] + 1, x2_span[0]]
            output_left  = self.neuron.predict(input=input_left,  iteration=iteration)
            output_right = self.neuron.predict(input=input_right, iteration=iteration)
            if output_left < output_right:
                color_left  = self.colors[0]
                color_right = self.colors[1]
            else:
                color_left  = self.colors[1]
                color_right = self.colors[0]
            colors_lr.append([color_left, color_right])
        x2_span_s              = sort(x2_span)
        self.class_input       = (array([x for x, y in zip(X, Y) if y == c]) for c in C)
        self.source1_weights   = ColumnDataSource(data=dict(x        = x1_span,                    y        = x2_span,
                                                            y1_left  = [x2_span_s[0], x2_span[1]], y2_left  = [x2_span_s[1], x2_span[1]],
                                                            y1_right = [x2_span[0], x2_span_s[0]], y2_right = [x2_span[0], x2_span_s[1]]),
                                                            tags     = [X1_span, X2_span, colors_lr])
        self.source1_selection = ColumnDataSource(data=dict(x=[X[-1,0]], y=[X[-1,1]]), tags=[X[:,0], X[:,1]])
        self.source2_iteration = ColumnDataSource(data=dict(x=I, y=L), tags=[[i for i in I], L])
        self.source2_epoch     = ColumnDataSource(data=dict(x=E, y=LE), tags=[[e for e in E], LE, k])
        self.x_range           = Range1d(*x1_range, bounds="auto")
        self.y_range           = Range1d(*x2_range, bounds="auto")

    def show(self):
        show(self.layout)

In [ ]:
class Figure_regression():
    def __init__(self, neuron, width=400, height=400):
        self.neuron = neuron
        self.__init_plot_params__()

#         [fig1, area_left, area_right] = self.__create_figure1__()
#         fig2   = self.__create_figure2__()
#         slider = self.__create_slider__(area_left, area_right)
#         if slider is None:
#             self.layout = row(fig1, fig2)
#         else:
#             self.layout = column(slider, row(fig1, fig2))
        fig1 = self.__create_figure1__()
        slider = self.__create_slider__()
        if slider is None:
            self.layout = fig1
        else:
            self.layout = column(slider, fig1)

    def __create_figure1__(self, width=400, height=400):
#         colors_lr  = self.source1_weights.tags[2][-1]
        figure     = Figure(plot_width=width, plot_height=height)
#         area_left  = figure.varea(x='x', y1='y1_left',  y2='y2_left',  source=self.source1_weights, fill_color=colors_lr[0], fill_alpha=0.2)
#         area_right = figure.varea(x='x', y1='y1_right', y2='y2_right', source=self.source1_weights, fill_color=colors_lr[1], fill_alpha=0.2)
#         for class_input, legend_label, color in zip(self.class_input, self.legend_labels, self.colors):
#             x = class_input[:,0]
#             y = class_input[:,1]
#             c = figure.circle(x, y, size=10, fill_color=color, fill_alpha=0.6, line_color=None, legend_label=legend_label)
        figure.line('x', 'y', source=self.source1, line_width=3, line_alpha=0.6, line_color='black', legend_label='Input')
        figure.line('x', 'yi', source=self.source1, line_width=3, line_alpha=0.6, line_color='red', legend_label='Output')
#         figure.circle('x', 'y', source=self.source1_selection, size=10, fill_color=None, line_color='black', line_alpha=0.6, line_width=3)
        # Properties
        figure.x_range = self.x_range
        figure.y_range = self.y_range
        figure.legend.click_policy = 'hide'
#         return figure, area_left, area_right
        return figure

#     def __create_figure2__(self, width=400, height=400):
#         figure = Figure(plot_width=width, plot_height=height, x_axis_label='Iteration', y_axis_label='Loss')
#         # Iteration axis
#         x_iteration = self.source2_iteration.data['x']
#         y_iteration = self.source2_iteration.data['y']
#         figure.line(x_iteration, y_iteration, line_width=3, line_color='black', line_alpha=0.1)
#         figure.line('x', 'y', source=self.source2_iteration, line_width=3, line_color='black', legend_label='Iteration loss')
#         # Epoch axis
#         x_epoch = self.source2_epoch.data['x']
#         y_epoch = self.source2_epoch.data['y']
#         color_epoch = 'red'
#         figure.extra_x_ranges = {'epoch': Range1d(start=0, end=self.neuron.epochs)}
#         figure.line(x_epoch, y_epoch, x_range_name='epoch', line_width=3, line_color=color_epoch, line_alpha=0.1)
#         figure.line('x', 'y', source=self.source2_epoch, x_range_name='epoch', line_width=3, line_color=color_epoch, legend_label='Epoch loss')
#         figure.add_layout(LinearAxis(x_range_name           = 'epoch',
#                                      axis_label             = 'Epoch',
#                                      axis_label_text_color  = color_epoch,
#                                      axis_line_color        = color_epoch,
#                                      major_label_text_color = color_epoch,
#                                      major_tick_line_color  = color_epoch,
#                                      minor_tick_line_color  = color_epoch),
#                           'above')
#         # Properties
#         if min(x_iteration) != max(x_iteration):
#             figure.x_range = Range1d(min(x_iteration), max(x_iteration), bounds="auto")
#         else:
#             figure.x_range = Range1d(min(x_iteration), max(x_iteration) + 1, bounds="auto")
#         figure.y_range = Range1d(0, max(y_iteration) + 1, bounds="auto")
#         figure.xaxis.ticker.min_interval = 1
#         figure.yaxis.ticker.min_interval = 1
#         figure.xaxis.ticker.num_minor_ticks = 0
#         figure.yaxis.ticker.num_minor_ticks = 0
#         figure.legend.click_policy = 'hide'
#         return figure

    def __create_slider__(self):
        if self.neuron.iterations == 0:
            return None
        else:
            callback1 = CustomJS(args=dict(source=self.source1), code="""
                var i             = cb_obj.value
                var yi            = source.tags[0][i]
                source.data['yi'] = yi
                source.change.emit()
            """)
#             callback1_selection = CustomJS(args=dict(source=self.source1_selection), code="""
#                 var i            = cb_obj.value
#                 var X            = source.tags[0]
#                 var Y            = source.tags[1]
#                 source.data['x'] = [X[i % X.length]]
#                 source.data['y'] = [Y[i % Y.length]]
#                 source.change.emit()
#             """)
#             callback2_iteration = CustomJS(args=dict(source=self.source2_iteration), code="""
#                 var i            = cb_obj.value
#                 var x            = source.tags[0].slice(0, i + 1)
#                 var y            = source.tags[1].slice(0, i + 1)
#                 source.data['x'] = x
#                 source.data['y'] = y
#                 source.change.emit()
#             """)
#             callback2_epoch = CustomJS(args=dict(source=self.source2_epoch), code="""
#                 var i    = cb_obj.value
#                 var end  = cb_obj.end
#                 var base = source.tags[2]
#                 if (i !== end)
#                 {
#                     var e = Math.floor(i / base)
#                 }
#                 else
#                 {
#                     var e = Math.ceil(i / base)
#                 }
#                 var x            = source.tags[0].slice(0, e + 1)
#                 var y            = source.tags[1].slice(0, e + 1)
#                 source.data['x'] = x
#                 source.data['y'] = y
#                 source.change.emit()
#             """)
#             slider = Slider(start=0, end=3, value=3, step=1, title='Iteration')
            slider = Slider(start=0, end=self.neuron.iterations, value=self.neuron.iterations, step=1, title='Iteration')
            slider.js_on_change('value', callback1)
#             slider.js_on_change('value', callback1_selection)
#             slider.js_on_change('value', callback2_iteration)
#             slider.js_on_change('value', callback2_epoch)
            return slider

    def __init_plot_params__(self):
#         n  = self.neuron.input_length
#         k  = self.neuron.num_inputs
        X  = self.neuron.inputs
        Y  = self.neuron.labels
        Xt = transpose(X)[0]
#         print('X',X)
#         print('Xt',Xt)
#         C  = self.neuron.unique_labels
#         W  = self.neuron.weights
        I  = range(0, self.neuron.iterations+1)
#         I  = range(0, self.neuron.iterations+1)[0:3]
#         E  = range(0, self.neuron.epochs+1)
#         L  = self.neuron.losses
#         LE = L[::k]
#         LE.append(L[-1])

#         self.colors        = ['blue', 'red']
#         self.legend_labels = ['Class ' + str(int(label)) for label in self.neuron.unique_labels]

        xmin, xmax = min(Xt), max(Xt)
        ymin, ymax = min(Y), max(Y)
        dy         = ymax - ymin
        x_range    = [xmin, xmax]
        y_range    = [ymin - 0.1*dy, ymax + 0.1*dy]
#         X1_span      = []
#         X2_span      = []
#         colors_lr    = []
#         for iteration in I:
#             x1_span = array(x1_range)
#             x2_span = array(self.neuron.kernel(input_class1=x1_span, iteration=iteration))
#             i       = argsort(x2_span)
#             if x2_span[i[0]] > x2_range[0] or x2_span[i[1]] < x2_range[1]:
#                 if x2_span[i[0]] > x2_range[0]:
#                     x2_span[i[0]] = x2_range[0]
#                 if x2_span[i[1]] < x2_range[1]:
#                     x2_span[i[1]] = x2_range[1]
#                 x1_span = array(self.neuron.kernel(input_class2=x2_span, iteration=iteration))
#             X1_span.append(x1_span)
#             X2_span.append(x2_span)
        YI = []
        for iteration in I:
            yi = array(self.neuron.predict(X, iteration=iteration))
            YI.append(yi)

#             input_left   = [x1_span[0] - 1, x2_span[0]]
#             input_right  = [x1_span[0] + 1, x2_span[0]]
#             output_left  = self.neuron.predict(input=input_left,  iteration=iteration)
#             output_right = self.neuron.predict(input=input_right, iteration=iteration)

#             if output_left < output_right:
#                 color_left  = self.colors[0]
#                 color_right = self.colors[1]
#             else:
#                 color_left  = self.colors[1]
#                 color_right = self.colors[0]
#             colors_lr.append([color_left, color_right])
#         x2_span_s = sort(x2_span)

#         self.class_input       = (array([x for x, y in zip(X, Y) if y == c]) for c in C)
        self.source1 = ColumnDataSource(data=dict(x=Xt, y=Y, yi=YI[-1]), tags=[YI])
#         self.source1_selection = ColumnDataSource(data=dict(x=[X[-1,0]], y=[X[-1,1]]), tags=[X[:,0], X[:,1]])
#         self.source2_iteration = ColumnDataSource(data=dict(x=I, y=L), tags=[[i for i in I], L])
#         self.source2_epoch     = ColumnDataSource(data=dict(x=E, y=LE), tags=[[e for e in E], LE, k])
        self.x_range = Range1d(*x_range, bounds="auto")
        self.y_range = Range1d(*y_range, bounds="auto")

    def show(self):
        show(self.layout)

# Train ADALINE and plot results
The dataset for ADALINE is a long list of number pairs.

...

In [ ]:
inputs3_1, labels3_1 = load_data3('./datasets/item 3/dataset_regression.csv')
display_data(inputs3_1, labels3_1, nrows=3)

adaline3_1 = Adaline1(input_length=ndims(inputs3_1), learning_rate=0.001, max_epochs=1e2)
adaline3_1.train(inputs3_1, labels3_1)

figure3_1 = Figure_regression(adaline3_1)
figure3_1.show()

In [ ]:
# inputs3_2 = insert(inputs3_1, 1, values=transpose(inputs3_1**2), axis=1)
# labels3_2 = labels3_1
# display_data(inputs3_2, labels3_2, nrows=3)
inputs3_1, labels3_1 = inputs3_2, labels3_2

adaline3_2 = Adaline2(input_length=ndims(inputs3_2), learning_rate=0.001, max_epochs=1e2)
adaline3_2.train(inputs3_2, labels3_2)

display_data(adaline3_2.inputs, adaline3_2.labels, nrows=3)

figure3_2 = Figure_regression(adaline3_2)
figure3_2.show()

In [ ]:
print(adaline3_2.labels)
adaline3_2.predict(0)

In [ ]:
raise SystemExit("Stop!")

# Train perceptron and plot results
## Dataset 1
In this dataset, the perceptron was able to converge pretty quickly with the following dataset.
The adaptation rule lead to the maximum minimum. That is because the dataset has classes quick separated from each other.

Note that the learning rate does not need to be quite small. One can play with the parameters, like the learning rate and maximum epochs to check different results. It can be seen that with greater learning rates, the perceptron is able to converge and faster with this dataset.

_(Hint: the iteration slider can be controlled by the keyboard.)_

In [ ]:
inputs1_1, labels1_1 = load_data1('./datasets/item 1/dataset1.txt')
inputs1_1, labels1_1 = keep_only_labels2(inputs1_1, labels1_1)
display_data(inputs1_1, labels1_1, nrows=3)

perceptron1_1 = Perceptron(input_length=ndims(inputs1_1), learning_rate=0.001, max_epochs=5e2)
perceptron1_1.train(inputs1_1, labels1_1)

figure1_1 = Figure_classification(perceptron1_1)
figure1_1.show()

## Dataset 2
This time, the dataset variance overlaps on the classes. That means points from one class can mix with points from other classes.
Since the perceptron can only separate classes linearly, the algorithm was not able to converge, stopping only after reaching the maximum number of epochs.

In [ ]:
inputs1_2, labels1_2 = load_data1('./datasets/item 1/dataset2.txt')
inputs1_2, labels1_2 = keep_only_labels2(inputs1_2, labels1_2)
display_data(inputs1_2, labels1_2, nrows=3)

perceptron1_2 = Perceptron(input_length=ndims(inputs1_2), learning_rate=0.0001, max_epochs=2000)
perceptron1_2.train(inputs1_2, labels1_2)

figure1_2 = Figure_classification(perceptron1_2)
figure1_2.show()

## Dataset 3
This dataset has concurrent classes mean positions and variances. The overlap and non-linearity of the data lead to an oscillating results, forbiding the perceptron to converge.

In [ ]:
inputs1_3, labels1_3 = load_data1('./datasets/item 1/dataset3.txt')
inputs1_3, labels1_3 = keep_only_labels2(inputs1_3, labels1_3)
display_data(inputs1_3, labels1_3, nrows=3)

perceptron1_3 = Perceptron(input_length=ndims(inputs1_3), learning_rate=0.001)
perceptron1_3.train(inputs1_3, labels1_3)

figure1_3 = Figure_classification(perceptron1_3)
figure1_3.show()

## Dataset 4
This dataset has a data disposition similar to the previous one, but with higher variance. Again, the overlap and non-linearity of the data lead to an oscillating results, forbiding the perceptron to converge.

In [ ]:
inputs1_4, labels1_4 = load_data1('./datasets/item 1/dataset4.txt')
inputs1_4, labels1_4 = keep_only_labels2(inputs1_4, labels1_4)
display_data(inputs1_4, labels1_4, nrows=3)

perceptron1_4 = Perceptron(input_length=ndims(inputs1_4), learning_rate=0.001)
perceptron1_4.train(inputs1_4, labels1_4)

figure1_4 = Figure_classification(perceptron1_4)
figure1_4.show()

## Dataset 5
This time, the dataset almost overlap, but the perceptron was able to converge, within given maximum number of epochs, since a linear separation of the dataset is possible.

In [ ]:
inputs1_5, labels1_5 = load_data1('./datasets/item 1/dataset5.txt')
inputs1_5, labels1_5 = keep_only_labels2(inputs1_5, labels1_5)
display_data(inputs1_5, labels1_5, nrows=3)

perceptron1_5 = Perceptron(input_length=ndims(inputs1_5), learning_rate=0.001, max_epochs=5e2)
perceptron1_5.train(inputs1_5, labels1_5, shuffle_data=True)

figure1_5 = Figure_classification(perceptron1_5)
figure1_5.show()

## Dataset 6
The following data has a similar disposition from the last one, but with higher spreadness. The result separation from the perceptron tried to find the best position between the centroids of both classes. However, it got stuck in near a local minimum solution, oscillating in loss due to the variance of both classes.

What is interesting to notice is that after around 300 epochs, the oscillation in loss increased. That demonstrate the instability of the perceptron to keep stuck around a local minimum.

In [ ]:
inputs1_6, labels1_6 = load_data1('./datasets/item 1/dataset6.txt')
inputs1_6, labels1_6 = keep_only_labels2(inputs1_6, labels1_6)
display_data(inputs1_6, labels1_6, nrows=3)

perceptron1_6 = Perceptron(input_length=ndims(inputs1_6), learning_rate=0.001, max_epochs=5e2)
perceptron1_6.train(inputs1_6, labels1_6, shuffle_data=True)

figure1_6 = Figure_classification(perceptron1_6)
figure1_6.show()